# REINFORCE

---

In this notebook, we will train REINFORCE with OpenAI Gym's Cartpole environment.

### 1. Import the Necessary Packages

In [1]:
import gym
gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### 2. Define the Architecture of the Policy

In [2]:
env = gym.make('CartPole-v0')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

observation space: Box(4,)
action space: Discrete(2)


### 3. Train the Agent with REINFORCE

In [3]:
policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()
        
        #print(saved_log_probs, R, policy_loss)
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
    
scores = reinforce()

[tensor([-0.5285], grad_fn=<SqueezeBackward1>), tensor([-0.9241], grad_fn=<SqueezeBackward1>), tensor([-0.5270], grad_fn=<SqueezeBackward1>), tensor([-0.5042], grad_fn=<SqueezeBackward1>), tensor([-0.9514], grad_fn=<SqueezeBackward1>), tensor([-0.9307], grad_fn=<SqueezeBackward1>), tensor([-0.5221], grad_fn=<SqueezeBackward1>), tensor([-0.5007], grad_fn=<SqueezeBackward1>), tensor([-0.9567], grad_fn=<SqueezeBackward1>), tensor([-0.4985], grad_fn=<SqueezeBackward1>), tensor([-0.4827], grad_fn=<SqueezeBackward1>), tensor([-0.9836], grad_fn=<SqueezeBackward1>), tensor([-0.4794], grad_fn=<SqueezeBackward1>), tensor([-0.4691], grad_fn=<SqueezeBackward1>), tensor([-0.4734], grad_fn=<SqueezeBackward1>), tensor([-0.4794], grad_fn=<SqueezeBackward1>), tensor([-0.4861], grad_fn=<SqueezeBackward1>), tensor([-0.9435], grad_fn=<SqueezeBackward1>)] 18.0 tensor(209.5441, grad_fn=<SumBackward0>)
[tensor([-0.9513], grad_fn=<SqueezeBackward1>), tensor([-0.8962], grad_fn=<SqueezeBackward1>), tensor([-0.8

[tensor([-0.5215], grad_fn=<SqueezeBackward1>), tensor([-0.9466], grad_fn=<SqueezeBackward1>), tensor([-0.9031], grad_fn=<SqueezeBackward1>), tensor([-0.8329], grad_fn=<SqueezeBackward1>), tensor([-0.7583], grad_fn=<SqueezeBackward1>), tensor([-0.6719], grad_fn=<SqueezeBackward1>), tensor([-0.7509], grad_fn=<SqueezeBackward1>), tensor([-0.7088], grad_fn=<SqueezeBackward1>), tensor([-0.7211], grad_fn=<SqueezeBackward1>), tensor([-0.6874], grad_fn=<SqueezeBackward1>), tensor([-0.6560], grad_fn=<SqueezeBackward1>), tensor([-0.7860], grad_fn=<SqueezeBackward1>), tensor([-0.7266], grad_fn=<SqueezeBackward1>), tensor([-0.7021], grad_fn=<SqueezeBackward1>), tensor([-0.7183], grad_fn=<SqueezeBackward1>), tensor([-0.7115], grad_fn=<SqueezeBackward1>), tensor([-0.6784], grad_fn=<SqueezeBackward1>), tensor([-0.6419], grad_fn=<SqueezeBackward1>), tensor([-0.5913], grad_fn=<SqueezeBackward1>), tensor([-0.5373], grad_fn=<SqueezeBackward1>), tensor([-0.9369], grad_fn=<SqueezeBackward1>), tensor([-0.5

[tensor([-0.7936], grad_fn=<SqueezeBackward1>), tensor([-0.6129], grad_fn=<SqueezeBackward1>), tensor([-0.6001], grad_fn=<SqueezeBackward1>), tensor([-0.7947], grad_fn=<SqueezeBackward1>), tensor([-0.7998], grad_fn=<SqueezeBackward1>), tensor([-0.6116], grad_fn=<SqueezeBackward1>), tensor([-0.5952], grad_fn=<SqueezeBackward1>), tensor([-0.8039], grad_fn=<SqueezeBackward1>), tensor([-0.5920], grad_fn=<SqueezeBackward1>), tensor([-0.5891], grad_fn=<SqueezeBackward1>), tensor([-0.5911], grad_fn=<SqueezeBackward1>), tensor([-0.8056], grad_fn=<SqueezeBackward1>), tensor([-0.5849], grad_fn=<SqueezeBackward1>), tensor([-0.8127], grad_fn=<SqueezeBackward1>), tensor([-0.8202], grad_fn=<SqueezeBackward1>), tensor([-0.5753], grad_fn=<SqueezeBackward1>), tensor([-0.5785], grad_fn=<SqueezeBackward1>), tensor([-0.8195], grad_fn=<SqueezeBackward1>), tensor([-0.8257], grad_fn=<SqueezeBackward1>), tensor([-0.8307], grad_fn=<SqueezeBackward1>), tensor([-0.8296], grad_fn=<SqueezeBackward1>), tensor([-0.5

[tensor([-0.7268], grad_fn=<SqueezeBackward1>), tensor([-0.7516], grad_fn=<SqueezeBackward1>), tensor([-0.6375], grad_fn=<SqueezeBackward1>), tensor([-0.6380], grad_fn=<SqueezeBackward1>), tensor([-0.7326], grad_fn=<SqueezeBackward1>), tensor([-0.7537], grad_fn=<SqueezeBackward1>), tensor([-0.6384], grad_fn=<SqueezeBackward1>), tensor([-0.6348], grad_fn=<SqueezeBackward1>), tensor([-0.7438], grad_fn=<SqueezeBackward1>), tensor([-0.6315], grad_fn=<SqueezeBackward1>), tensor([-0.6375], grad_fn=<SqueezeBackward1>), tensor([-0.6588], grad_fn=<SqueezeBackward1>), tensor([-0.6844], grad_fn=<SqueezeBackward1>), tensor([-0.7090], grad_fn=<SqueezeBackward1>), tensor([-0.6408], grad_fn=<SqueezeBackward1>), tensor([-0.6902], grad_fn=<SqueezeBackward1>), tensor([-0.6624], grad_fn=<SqueezeBackward1>), tensor([-0.6983], grad_fn=<SqueezeBackward1>), tensor([-0.6554], grad_fn=<SqueezeBackward1>), tensor([-0.6816], grad_fn=<SqueezeBackward1>), tensor([-0.7157], grad_fn=<SqueezeBackward1>), tensor([-0.7

[tensor([-0.6944], grad_fn=<SqueezeBackward1>), tensor([-0.7483], grad_fn=<SqueezeBackward1>), tensor([-0.8035], grad_fn=<SqueezeBackward1>), tensor([-0.5412], grad_fn=<SqueezeBackward1>), tensor([-0.5896], grad_fn=<SqueezeBackward1>), tensor([-0.6304], grad_fn=<SqueezeBackward1>), tensor([-0.6764], grad_fn=<SqueezeBackward1>), tensor([-0.7214], grad_fn=<SqueezeBackward1>), tensor([-0.6399], grad_fn=<SqueezeBackward1>), tensor([-0.7120], grad_fn=<SqueezeBackward1>), tensor([-0.7418], grad_fn=<SqueezeBackward1>), tensor([-0.6210], grad_fn=<SqueezeBackward1>), tensor([-0.6537], grad_fn=<SqueezeBackward1>), tensor([-0.6973], grad_fn=<SqueezeBackward1>), tensor([-0.7295], grad_fn=<SqueezeBackward1>), tensor([-0.6316], grad_fn=<SqueezeBackward1>), tensor([-0.6626], grad_fn=<SqueezeBackward1>), tensor([-0.6890], grad_fn=<SqueezeBackward1>), tensor([-0.7223], grad_fn=<SqueezeBackward1>), tensor([-0.6373], grad_fn=<SqueezeBackward1>), tensor([-0.6674], grad_fn=<SqueezeBackward1>), tensor([-0.7

...

[tensor([-0.4537], grad_fn=<SqueezeBackward1>), tensor([-0.6951], grad_fn=<SqueezeBackward1>), tensor([-0.4603], grad_fn=<SqueezeBackward1>), tensor([-0.6890], grad_fn=<SqueezeBackward1>), tensor([-1.0911], grad_fn=<SqueezeBackward1>), tensor([-0.1556], grad_fn=<SqueezeBackward1>), tensor([-0.3715], grad_fn=<SqueezeBackward1>), tensor([-0.6374], grad_fn=<SqueezeBackward1>), tensor([-0.9315], grad_fn=<SqueezeBackward1>), tensor([-0.3108], grad_fn=<SqueezeBackward1>), tensor([-0.9982], grad_fn=<SqueezeBackward1>), tensor([-0.2766], grad_fn=<SqueezeBackward1>), tensor([-0.4027], grad_fn=<SqueezeBackward1>), tensor([-0.8227], grad_fn=<SqueezeBackward1>), tensor([-0.3525], grad_fn=<SqueezeBackward1>), tensor([-0.9283], grad_fn=<SqueezeBackward1>), tensor([-0.2974], grad_fn=<SqueezeBackward1>), tensor([-1.0680], grad_fn=<SqueezeBackward1>), tensor([-0.2405], grad_fn=<SqueezeBackward1>), tensor([-1.2511], grad_fn=<SqueezeBackward1>), tensor([-1.7780], grad_fn=<SqueezeBackward1>), tensor([-0.1

[tensor([-0.5759], grad_fn=<SqueezeBackward1>), tensor([-0.4862], grad_fn=<SqueezeBackward1>), tensor([-0.6449], grad_fn=<SqueezeBackward1>), tensor([-1.0374], grad_fn=<SqueezeBackward1>), tensor([-0.2520], grad_fn=<SqueezeBackward1>), tensor([-0.3650], grad_fn=<SqueezeBackward1>), tensor([-0.8889], grad_fn=<SqueezeBackward1>), tensor([-0.3075], grad_fn=<SqueezeBackward1>), tensor([-1.0293], grad_fn=<SqueezeBackward1>), tensor([-0.2514], grad_fn=<SqueezeBackward1>), tensor([-0.3595], grad_fn=<SqueezeBackward1>), tensor([-0.8282], grad_fn=<SqueezeBackward1>), tensor([-0.2974], grad_fn=<SqueezeBackward1>), tensor([-1.0081], grad_fn=<SqueezeBackward1>), tensor([-1.5412], grad_fn=<SqueezeBackward1>), tensor([-0.1528], grad_fn=<SqueezeBackward1>), tensor([-0.1863], grad_fn=<SqueezeBackward1>), tensor([-0.2571], grad_fn=<SqueezeBackward1>), tensor([-1.0674], grad_fn=<SqueezeBackward1>), tensor([-0.2074], grad_fn=<SqueezeBackward1>), tensor([-0.3028], grad_fn=<SqueezeBackward1>), tensor([-0.5

[tensor([-0.2821], grad_fn=<SqueezeBackward1>), tensor([-0.5429], grad_fn=<SqueezeBackward1>), tensor([-0.8224], grad_fn=<SqueezeBackward1>), tensor([-0.3560], grad_fn=<SqueezeBackward1>), tensor([-0.5414], grad_fn=<SqueezeBackward1>), tensor([-0.8016], grad_fn=<SqueezeBackward1>), tensor([-1.3622], grad_fn=<SqueezeBackward1>), tensor([-0.1044], grad_fn=<SqueezeBackward1>), tensor([-1.3874], grad_fn=<SqueezeBackward1>), tensor([-0.0995], grad_fn=<SqueezeBackward1>), tensor([-1.4396], grad_fn=<SqueezeBackward1>), tensor([-0.0920], grad_fn=<SqueezeBackward1>), tensor([-0.2472], grad_fn=<SqueezeBackward1>), tensor([-0.5219], grad_fn=<SqueezeBackward1>), tensor([-0.5945], grad_fn=<SqueezeBackward1>), tensor([-0.8787], grad_fn=<SqueezeBackward1>), tensor([-0.2457], grad_fn=<SqueezeBackward1>), tensor([-0.8798], grad_fn=<SqueezeBackward1>), tensor([-0.2443], grad_fn=<SqueezeBackward1>), tensor([-0.8792], grad_fn=<SqueezeBackward1>), tensor([-0.2436], grad_fn=<SqueezeBackward1>), tensor([-0.5

[tensor([-0.4810], grad_fn=<SqueezeBackward1>), tensor([-0.8233], grad_fn=<SqueezeBackward1>), tensor([-1.2428], grad_fn=<SqueezeBackward1>), tensor([-0.1845], grad_fn=<SqueezeBackward1>), tensor([-0.2844], grad_fn=<SqueezeBackward1>), tensor([-1.0466], grad_fn=<SqueezeBackward1>), tensor([-1.5619], grad_fn=<SqueezeBackward1>), tensor([-2.1258], grad_fn=<SqueezeBackward1>), tensor([-0.0826], grad_fn=<SqueezeBackward1>), tensor([-0.1005], grad_fn=<SqueezeBackward1>), tensor([-0.1220], grad_fn=<SqueezeBackward1>), tensor([-0.1523], grad_fn=<SqueezeBackward1>), tensor([-0.2110], grad_fn=<SqueezeBackward1>), tensor([-0.3313], grad_fn=<SqueezeBackward1>), tensor([-0.5620], grad_fn=<SqueezeBackward1>), tensor([-1.0148], grad_fn=<SqueezeBackward1>), tensor([-0.1898], grad_fn=<SqueezeBackward1>), tensor([-0.7740], grad_fn=<SqueezeBackward1>), tensor([-0.2657], grad_fn=<SqueezeBackward1>), tensor([-0.5975], grad_fn=<SqueezeBackward1>), tensor([-1.2122], grad_fn=<SqueezeBackward1>), tensor([-0.1

KeyboardInterrupt: 

### 4. Plot the Scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch a Smart Agent!

In [ ]:
env = gym.make('CartPole-v0')

state = env.reset()
for t in range(1000):
    action, _ = policy.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()